In [10]:
import numpy as np
import scipy.io as sio
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from tensorflow.keras import layers, models, Model
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_curve, auc
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Load data
file_path = 'C:\\Users\\UC\\Documents\\NeuMa\\22117124\\new.mat'
new = sio.loadmat(file_path)
ET = new['ET']
Label_array_ET = new['label_list'].ravel()

print(f"ET shape: {ET.shape}, Label shape: {Label_array_ET.shape}")

# Create an empty list to store the images
image_list = []

# Loop through each sample
for sample_idx in range(ET.shape[2]):
    # Extract X and Y coordinates for both eyes
    x_left, y_left, _, x_right, y_right, _ = ET[:, :, sample_idx]

    # Create a blank canvas (64x64) for the gaze plot
    gaze_plot = np.zeros((64, 64, 3), dtype=np.uint8)  # Initialize as black image

    # Scale the coordinates to match the canvas size
    scaled_x_left = (x_left * 64).astype(int)
    scaled_y_left = (y_left * 64).astype(int)
    scaled_x_right = (x_right * 64).astype(int)
    scaled_y_right = (y_right * 64).astype(int)

    # Set gaze points as white pixels on the canvas
    for i in range(120):
        if 0 <= scaled_x_left[i] < 64 and 0 <= scaled_y_left[i] < 64:
            gaze_plot[scaled_y_left[i], scaled_x_left[i]] = [255, 255, 255]  # White color
        if 0 <= scaled_x_right[i] < 64 and 0 <= scaled_y_right[i] < 64:
            gaze_plot[scaled_y_right[i], scaled_x_right[i]] = [255, 255, 255]  # White color

    # Append the gaze plot to the image list
    image_list.append(gaze_plot)

# Convert the list of images to a NumPy array
image_array = np.array(image_list)

# Reshape images to have a single channel (grayscale)
image_array = np.expand_dims(image_array, axis=-1)

# Balance the data using SMOTE
smote = SMOTE(random_state=42)
image_array_flat = image_array.reshape(-1, 64*64*3)
image_array_resampled, Label_array_ET_resampled = smote.fit_resample(image_array_flat, Label_array_ET)

# Reshape the resampled data back to the original shape
image_array_resampled = image_array_resampled.reshape(-1, 64, 64, 3)

# Extract features using a pre-trained VGG16 model
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))
vgg_model = Model(inputs=vgg_model.input, outputs=vgg_model.get_layer('block5_pool').output)

# Normalize the data for VGG16
image_array_resampled = preprocess_input(image_array_resampled.astype(np.float32))

# Extract features
features = vgg_model.predict(image_array_resampled)

# Flatten the features to be combined with statistical features
features = features.reshape(features.shape[0], -1)

# Ensure the statistical features are computed correctly
ET_resampled = np.array([ET[:, :, i % ET.shape[2]] for i in range(image_array_resampled.shape[0])])

# Enhanced statistical feature extraction
stat_features = []
for sample_idx in range(ET_resampled.shape[2]):
    # Extract X and Y coordinates for both eyes
    x_left = ET_resampled[0, :, sample_idx]
    y_left = ET_resampled[1, :, sample_idx]
    pupil_left = ET_resampled[2, :, sample_idx]
    x_right = ET_resampled[3, :, sample_idx]
    y_right = ET_resampled[4, :, sample_idx]
    pupil_right = ET_resampled[5, :, sample_idx]

    # Compute statistical features
    # Fixation points: Mean and variance of X and Y coordinates
    fixation_features = [
        np.mean(x_left), np.var(x_left),
        np.mean(y_left), np.var(y_left),
        np.mean(x_right), np.var(x_right),
        np.mean(y_right), np.var(y_right)
    ]
    
    # Pupil dilation: Mean and variance of pupil sizes
    pupil_features = [
        np.mean(pupil_left), np.var(pupil_left),
        np.mean(pupil_right), np.var(pupil_right)
    ]
    
    # Saccades: Mean and variance of differences between consecutive X and Y coordinates
    saccade_features = [
        np.mean(np.diff(x_left)), np.var(np.diff(x_left)),
        np.mean(np.diff(y_left)), np.var(np.diff(y_left)),
        np.mean(np.diff(x_right)), np.var(np.diff(x_right)),
        np.mean(np.diff(y_right)), np.var(np.diff(y_right))
    ]
    
    # Combine all features
    features_list = fixation_features + pupil_features + saccade_features
    stat_features.append(features_list)

stat_features = np.array(stat_features)

# Combine deep learning features with statistical features
combined_features = np.hstack([features, stat_features])

# Normalize combined features
scaler = StandardScaler()
combined_features = scaler.fit_transform(combined_features)

# Split the data into training, validation, and test sets
x_train, x_temp, y_train, y_temp = train_test_split(combined_features, Label_array_ET_resampled, test_size=0.3, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

# Train a Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(x_train, y_train)

# Evaluate the Random Forest classifier on the validation set
val_pred_rf = rf_classifier.predict(x_val)
val_acc_rf = accuracy_score(y_val, val_pred_rf)
print(f"Random Forest Validation accuracy: {val_acc_rf:.4f}")

# Evaluate the Random Forest classifier on the test set
test_pred_rf = rf_classifier.predict(x_test)
test_acc_rf = accuracy_score(y_test, test_pred_rf)
print(f"Random Forest Test accuracy: {test_acc_rf:.4f}")

# Train an XGBoost classifier
xgb_classifier = XGBClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
xgb_classifier.fit(x_train, y_train)

# Evaluate the XGBoost classifier on the validation set
val_pred_xgb = xgb_classifier.predict(x_val)
val_acc_xgb = accuracy_score(y_val, val_pred_xgb


SyntaxError: unterminated string literal (detected at line 3) (915738250.py, line 3)